In [ ]:
def commit_logger(project_names):
    
    for directory in project_names:
#         if not os.path.exists(os.path.join(directory, ".git")):
#             print(directory)
#             continue  # skip if its not a git repository

        os.chdir(directory)  # move to repository directory

        # Edit to desired date
        start_date = "2021-10-01"
        end_date = "2023-07-18"  
        output_file = f"{directory}-log.json"

        # run logger.sh 
        # Pass start_date, end_date, output_file as standard input
        subprocess.run(['bash', '../logger.sh', start_date, end_date, output_file])
        
        os.chdir("..")
        print(directory, "commit logs are stored.")
        
    print("all project's log.json are successfully implemented")

# we won't use following function
def validate_log_json_file(project_names, output_directory):

    for directory in project_names:
        
        commits = []
        log_file = f"{directory}-log.json"
        
        os.chdir(directory)  # move to repository directory
        
        with open(log_file, "r") as f:
            try:
                log_data = json.load(f)
                commits.extend(log_data)
            except json.JSONDecodeError:
                combined_lines = ""
                # This is unfinished - let's do it manually...
        
        os.chdir("..")
        # Save the validated json file in "../{output_directory"/{log_file}"
    print("All json files have been validated.")
    

In [ ]:
import os
import json
import subprocess
    
def add_changed_file_list(project_names, output_directory):
    # Add and save the changed_file_list item to the created json file.
    # Save as new, leaving only the necessary items (hash, parent, date, subject, body, changed_file_list)
    
    for directory in project_names:
        commits = []
        
        log_file = os.path.join(os.getcwd(), output_directory, f"{directory}-git-log.json")
        with open(log_file, "r", encoding='utf-8') as f:
            data = f.read()
            raw_data = r''+data
            try:
                commits = json.loads(raw_data)
#                 print(commits)
            except json.JSONDecodeError as e:
                print(f"JSON decoding error: {e}")
        
        os.chdir(directory)
        
        cnt = 0
        for commit in commits:
            files = []
            java_files = []
            commit['changed_file_list'] = []
            
            changed_file_list = subprocess.run(['git', 'show', commit['commitHash'], '--name-only', '--pretty=format:%b'], capture_output=True, text=True)
            if changed_file_list.returncode == 0 and changed_file_list.stdout:
                lines = changed_file_list.stdout.split('\n')
                # Process the rest of the code using the 'lines' variable
            else:
                # Handle the case when the Git command didn't execute successfully
                print("Failed to get the changed file list of ", commit['commitHash'])
            
            for line in lines:
                if line.strip() in commit['body']:
                    continue
                elif line.strip() != '':
                    files.append(line.strip())

            java_files = [file for file in files if file.endswith('.java')]
            java_files = list(filter(lambda file: 'src/test' not in file, java_files))
            
            if java_files != []:
                commit['changed_file_list'] = java_files
                cnt += 1
                # print(commit['commitHash'])
                # print(cnt, ":", commit['changed_file_list'])
        
        os.chdir("..")
        new_log_file = os.path.join(output_directory, f"{directory}-files-log.json")
        with open(new_log_file, "w", encoding='utf-8') as nf:
            json.dump(commits, nf, indent=2)
        print(directory, "- log.json was updated with modified file list")
        
    print("All json files have been updated.")

In [ ]:
if __name__ == "__main__":
    # You can edit following directory lists.
    directories = ["h2database", "bc-java", "pgjdbc", "junit4", "gson", "guava"]
    output_directory = "commit-logs"
    # commit_logger(directories)
    add_changed_file_list(directories, output_directory)

In [ ]:
os.getcwd()

In [ ]:
os.chdir("..")